In [18]:
# conda environment: vllm
# 使用qwen2 7B总结摘要
import os
import json
import pandas as pd
import requests
from string import Template
from tqdm import tqdm

In [19]:
qwen2___5_7B_instruct = "http://0.0.0.0:7012/v1/chat/completions"

In [20]:
url = qwen2___5_7B_instruct

In [ ]:
def ask_qwen(user_input, url=url):
    request_template = {
        "model": "model",
        "stop": ["<|endoftext|>", "<|im_end|>"],
        "messages": [],
        "temperature": 0.3,#ds 0.5-0.6
        "top_p": 0.9,
        "top_k": 40,
        "repetition_penalty": 1.05,
    }

    request_template["messages"].append(
        {"role": "user", "content": user_input}
    )
    # 添加异常打印输出
    # response = requests.post(url=url, json=request_template).json()
    # content = response["choices"][0]["message"]["content"]
    # return content

    try:
        response = requests.post(url=url, json=request_template)
        response.raise_for_status()  # 确保HTTP状态码正常
        data = response.json()
        return data["choices"][0]["message"]["content"]
    except requests.exceptions.RequestException as e:
        print(f"Request failed for input: {user_input}\nError: {e}")
        return "输出错误"

In [22]:
prompt_file = '../prompts/abstract_v6.txt'

In [23]:
with open(prompt_file, 'r', encoding='utf-8') as file:
    prompt_template = file.read()
template = Template(prompt_template)
print(repr(template))

# 加载测试数据

In [24]:
# test_data_path = '/root/autodl-tmp/data/raw_data/yanbao_test_llama.json'
# train_data_path = '/root/autodl-tmp/data/raw_data/yanbao_train_llama.json'

test_data_path = '/root/autodl-tmp/data/raw_data/yanbao_test_llama_thuc.json'
train_data_path = '/root/autodl-tmp/data/raw_data/yanbao_train_llama_thuc.json'

In [26]:
def read_json(file):
    if not file.endswith('.json'):
        print("file format is wrong!!!")
    else:
        with open(file, 'r', encoding='utf-8') as test_file:
            data = []
            for line in test_file:
                data_json = json.loads(line)
                data.append(data_json)
        return data

In [27]:
test_data = read_json(test_data_path)
train_data = read_json(train_data_path)

In [28]:
test_df = pd.DataFrame(test_data)
train_df = pd.DataFrame(train_data)

## 局部测试

In [29]:
df = test_df.head(5)
summarys = []
for idx, row in tqdm(df.iterrows(), total=len(df)):
    print("==============================================")
    input = row["input"]
    prompt = template.substitute(
        input=input
    )
    summary = ask_qwen(prompt)
    summarys.append(summary)
    print("input:", row["input"])
    print("output:", row["output"])
    print("summary:", summary)
    print("================================================")

# ##匹配掉思考过程
# import re
# from tqdm import tqdm

# # 假设你已经有 test_df 数据框
# # df = test_df.head(5)
# summarys = []

# # 使用正则表达式的函数来提取和过滤内容
# def filter_summary(text):
#     return re.sub(r"(?<=summary:)(.*?)(?=</think>)", "", text, flags=re.DOTALL).strip()

# for idx, row in tqdm(df.iterrows(), total=len(df)):
#     print("==============================================")
#     input_text = row["input"]
    
#     # 使用你原来的模板生成prompt
#     prompt = template.substitute(input=input_text)
    
#     # 调用问答接口获取summary
#     summary = ask_qwen(prompt)
    
#     # 对summary进行过滤处理
#     filtered_summary = filter_summary(summary)
#     summarys.append(filtered_summary)
    
#     # 输出结果
#     print("input:", row["input"])
#     print("output:", row["output"])
#     print("summary:", filtered_summary)



  0%|          | 0/5 [00:00<?, ?it/s]

 20%|██        | 1/5 [00:04<00:16,  4.08s/it]

input: 9月26日消息高盛称将太古00019评级由买入降至中性目标价由128元调低至99.7元高盛将太古00019评级由买入降至中性认为分拆太古地产上市具战略价值但基于91%资产公开上市相信会加深资产净值折让目标价由128元调低至99.7元即较NAV折让由原先20%扩至34%报告指太古曾于2009年11月宣布分拆太古地产但最后因市场条件放弃高盛认为今年7月出售又一城及现时分拆建议策略明智因没有融资需求上市毋须根据市况进行加上不涉及发行新股亦不会摊薄太古持有股东权益同时配合太古长远扩张地产业务策略可为太古地产日后融资提供平台计及国泰00293评级早前被下调太古20112013年每股盈测调低2%18%现价较NAV折让43%相对过往平均24%估值看似不高但分拆及控股公司可能扩阔折让加上忧虑太古周期性业务受经济放缓影响投资价值不可能提升
output: 中性
summary: **摘要**：高盛将太古评级由买入降至中性，目标价由128元调低至99.7元，因分拆太古地产后NAV折让扩大至34%，较以往平均折让24%。尽管分拆策略明智，但经济放缓可能影响周期性业务，每股盈测下调2%-18%，未来前景不明朗。


 40%|████      | 2/5 [00:08<00:12,  4.05s/it]

input: 股指弱势震荡近期地产板块走势活跃如600736苏州高新根据广发证券投资报告苏州高新有望跨区发展文中提到借鉴华侨城集团的经营模式苏州高新也提出了旅游2B地产的经营模式与其他房地产公司相比公司的苏州乐园能够给当地城市带来更多商机因此更容易得到当地政府的支持虽然苏州乐园的规模无法与华侨城集团旗下的乐园相比但较小的规模也降低了公司跨区域经营的资金压力公司拥有多年经营乐园的经验因此这种经营模式的难点在于资金公司10亿元的公司债已获得证监会批准预计年内发行加上公司现有资金能够保证公司12个此类项目的开发预计今明两年公司主营业务收入分别为31亿元和37亿元每股收益分别为0.289元和0.338元公司当前估值低于市场平均水平与公司行业中等水平地位不太吻合给予公司买入的评级目标价为8.50元此外前期国泰君安等机构也给予苏州高新买入评级目标位13元更高当然机构的预测能否实现有待观察该股近期走势也非常强周五盘中下跌之际该股逆市抗跌尾盘随大盘反弹迅速拉起日K线形成两阳夹阴多方炮组合需要提醒投资者的是近期大盘震荡加剧我们点评苏州高新只是给大家一个选股思路就是把握机构调研报告中确有潜力的品种逢低吸纳不是建议大家追高买入所点评的品种
output: 买入
summary: **摘要**：苏州高新主营收入预计今明两年分别达31亿元和37亿元，每股收益分别为0.289元和0.338元，得益于旅游地产模式及政府支持。公司财务稳健，获10亿公司债批准，锁定12个项目开发。机构给予“买入”评级，目标价8.50元，但市场波动需关注。


...

input: 2009年公司实现营业收入7.02亿元比上年同期增加10.74%营业利润1.51亿元同比提升23.76%归属于母公司的净利润1.33亿元同比增幅为30.07%按期末股本全面摊薄后EPS0.74元不分配不转增收入的增加毛利率的提升尚未使用的上市募集资金增加了利息收入所得税率由18%下降到2009年的15%都是业绩增长的支撑因素从产品构成上看公司主导产品轻型包装产品的收入和毛利占50%多另有重型包装产品和第三方采购产品均在20%以上由于行业需求的快速增长以及公司牢牢占据高端客户09年公司综合毛利率达到32.01%净利率达到18.90%均比07年有较大幅度的提高随着业务规模的扩展毛利率水平不降反升确实反映了行业与公司的前景美盈森上市超募大量资金迅速在苏州和重庆布局设点加上原有在东莞的募投项目三个项目的设计产能将达到4.3亿平方米是现有产能的5倍多预计东莞项目10年底前投产苏州项目稍后投产重庆项目11年1季度末投产公司预期用三年的时间逐步消化新项目的产能即2013年完全达产我们认为随着宏观经济的复苏以及下游产业如电子通信家电食品等行业的持续快速发展瓦楞包装产业未来的发展同样可期公司层面随着募投项目的投产将进入收入盈利的爆发增长期预测201011年美盈森净利润分别增长28%63%EPS分别为0.95元1.55元我们看好纸包装行业及公司未来几年的高成长性维持增持评级目标价40元
output: 谨慎增持
summary: **摘要**：2009年美盈森营收同比增长10.74%，净利润增30.07%，得益于高端客户需求旺盛和毛利率提升。公司募投项目进展顺利，预计未来三年产能大幅扩张，推动业绩爆发式增长。维持“增持”评级，目标价40元，看好其高成长性。


 80%|████████  | 4/5 [00:15<00:03,  3.72s/it]

input: 华宝国际逆市跌0.42%报7.05港元成交1894.10万港元该股早盘跑输大市恒指涨2.52%摩根大通和大华继显对该股看法不摩通维持该股中性评级但目标价由6.20港元提升至7.30港元大华继显维持该股卖出评级虽上调目标价至6.40港元不过仍低于现价摩根大通称此前将该公司10财年的盈利预测上调2%将10财年的预测市盈率估值由原来的15倍提高至17倍较中国生活必需品类股的平均水平折让20%左右指出主要风险来自目前食用香精业务的前景不明朗大华继显指出2009财年每股收益比上年增长29%至11.15亿港元与预期一致维持201012财年净利润预期不变指出管理层预计未来3年每年的内部增长率20%该行预计两年内利润增长率将低于15%表示虽然预计201012财年每股收益的复合年增长率为16%但现价相当于2010财年市盈率16.5倍认为已充分反映了近期的增长前景因此维持卖出评级
output: 中性
summary: **摘要**：华宝国际逆市下跌0.42%，2009财年每股收益同比增长29%，但2010-2012财年净利润预期未变。摩根大通上调盈利预测，维持中性评级；大华继显维持卖出评级，因未来三年利润增长率预计低于15%。


100%|██████████| 5/5 [00:19<00:00,  3.91s/it]

input: 10月10日消息摩根士丹利把瑞年国际评级从增持下调至减持把目标价从7港元下调至1.90港元把20112013财年销量预期下调30%60%利润预期下调45%70%摩根士丹利指出瑞年国际在2008年下半年受金融危机影响而未能达到销售目标该行对销售商超市和药店进行的调查显示出这种状况可能再度发生该行补充说与3年前相比瑞年国际情况较好拥有更多现金作为缓冲但该行担心如果经济持续低迷则由2011年上半年营运资金上升导致的经营现金流为负状况将延续至2011年下半年或2012年上半年
output: 卖出
summary: **摘要**：摩根士丹利将瑞年国际评级从增持下调至减持，目标价降至1.90港元，预计2011-2013财年销量和利润分别下降30%-60%及45%-70%。受经济低迷影响，公司现金流转负，但拥有更多现金缓冲。未来若经济持续低迷，业绩恢复前景不明朗，存在较大市场风险。


###    开始测试

In [ ]:
# # 测试集遍历（单线程）
# summarys = []
# for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
#     input = row["input"]
#     prompt = template.substitute(
#         input=input
#     )
#     summary = ask_qwen(prompt)
#     summarys.append(summary)

In [ ]:
import multiprocess as mp
# 定义进程任务函数（多线程）
def process_row(row):
    input_text = row["input"]
    prompt = template.substitute(input=input_text)
    summary = ask_qwen(prompt)
    return summary

# 多进程处理
def process_with_multiprocessing(test_df, num_processes=8):
    with mp.Pool(processes=num_processes) as pool:
        # 使用 imap_unordered 进行并行处理并显示进度条
        summarys = list(tqdm(pool.imap(process_row, [row for _, row in test_df.iterrows()]), total=len(test_df)))
    return summarys

In [ ]:
import re
def extract_summary(text):
    """
    使用正则表达式提取各种格式下 "摘要" 后面的内容。
    支持以下情况：
    1. **摘要**：xxx
    2. 摘要：xxx
    3. ### 摘要 ###\nxxx
    4. 摘要:xxx
    5. 内容混合多处 "摘要" 的情况

    Args:
        text (str): 输入的文本内容。

    Returns:
        str: 提取到的摘要内容。如果没有找到，返回空字符串。
    """
    # 正则表达式匹配各种 "摘要" 格式
    patterns = [
        r"\*\*摘要\*\*[:：]\s*(.*?)\s*(?:\n|$)",  # **摘要**：xxx
        r"摘要[:：]\s*(.*?)\s*(?:\n|$)",              # 摘要：xxx or 摘要:xxx
        r"### 摘要 ###\s*(.*?)\s*(?:\n|$)",         # ### 摘要 ###\nxxx
    ]

    # 遍历匹配所有可能的格式
    for pattern in patterns:
        match = re.search(pattern, text, re.DOTALL)
        if match:
            return match.group(1).strip()

    # 如果未匹配到，则进一步尝试捕获行首的内容（兜底处理）
    lines = text.strip().split("\n")
    if lines:
        return lines[0].strip()

    return ""

In [ ]:
test_summarys = process_with_multiprocessing(test_df)
test_df["summary"] = [extract_summary(summary) for summary in test_summarys]
test_df["response"] = test_summarys

  5%|▍         | 81/1690 [02:06<41:52,  1.56s/it] Process ForkPoolWorker-8:
Process ForkPoolWorker-1:
Process ForkPoolWorker-6:
Process ForkPoolWorker-2:
Process ForkPoolWorker-5:
Process ForkPoolWorker-3:
Process ForkPoolWorker-7:
Process ForkPoolWorker-4:
Traceback (most recent call last):
  File "/root/miniconda3/envs/vllm/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/root/miniconda3/envs/vllm/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/root/miniconda3/envs/vllm/lib/python3.9/site-packages/multiprocess/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/root/miniconda3/envs/vllm/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (m

KeyboardInterrupt: 

In [ ]:
train_summarys = process_with_multiprocessing(train_df)
train_df["summary"] = [extract_summary(summary) for summary in train_summarys]
train_df["response"] = train_summarys

  0%|          | 34/13532 [06:02<39:58:26, 10.66s/it]Process ForkPoolWorker-14:
Process ForkPoolWorker-10:
Process ForkPoolWorker-13:
Process ForkPoolWorker-9:
Process ForkPoolWorker-15:
Process ForkPoolWorker-12:
Process ForkPoolWorker-16:
Process ForkPoolWorker-11:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/root/miniconda3/envs/vllm/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/root/miniconda3/envs/vllm/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/root/miniconda3/envs/vllm/lib/python3.9/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/root/miniconda3/envs/vllm/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/root/minicon

KeyboardInterrupt: 

  File "/root/miniconda3/envs/vllm/lib/python3.9/site-packages/requests/api.py", line 115, in post
    return request("post", url, data=data, json=json, **kwargs)
  File "/root/miniconda3/envs/vllm/lib/python3.9/site-packages/multiprocess/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/root/miniconda3/envs/vllm/lib/python3.9/site-packages/requests/api.py", line 59, in request
    return session.request(method=method, url=url, **kwargs)
  File "/tmp/ipykernel_1667/2958103174.py", line 6, in process_row
    summary = ask_qwen(prompt)


  File "/root/miniconda3/envs/vllm/lib/python3.9/site-packages/requests/api.py", line 115, in post
    return request("post", url, data=data, json=json, **kwargs)
  File "/tmp/ipykernel_1667/2958103174.py", line 6, in process_row
    summary = ask_qwen(prompt)
  File "/tmp/ipykernel_1667/3830561366.py", line 21, in ask_qwen
    response = requests.post(url=url, json=request_template)
  File "/tmp/ipykernel_1667/2958103174.py", line 6, in process_row
    summary = ask_qwen(prompt)
  File "/tmp/ipykernel_1667/2958103174.py", line 6, in process_row
    summary = ask_qwen(prompt)
  File "/root/miniconda3/envs/vllm/lib/python3.9/site-packages/requests/sessions.py", line 589, in request
    resp = self.send(prep, **send_kwargs)
  File "/root/miniconda3/envs/vllm/lib/python3.9/site-packages/requests/api.py", line 59, in request
    return session.request(method=method, url=url, **kwargs)
  File "/tmp/ipykernel_1667/3830561366.py", line 21, in ask_qwen
    response = requests.post(url=url, jso

In [ ]:
def save_different_formats(df, file_path):
    """
    Save DataFrame into three different JSONL formats based on specified rules.

    Args:
        df (pd.DataFrame): Input DataFrame with columns 'instruction', 'input', 'output', 'summary', 'response'.
        file_path (str): Base file path, including directory and the original file name.
    """
    # 1. Extract base file name and directory
    base_dir = os.path.dirname(file_path)
    base_name = os.path.splitext(os.path.basename(file_path))[0]

    # 如果文件夹不存在，则创建文件夹
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)

    # 2. Define output file names
    file1 = os.path.join(base_dir, f"{base_name}_raw_format.json")
    file2 = os.path.join(base_dir, f"{base_name}_with_instru_format.json")
    file3 = os.path.join(base_dir, f"{base_name}_wo_instru_format.json")

    # 3. Function to write JSONL file
    def write_jsonl(data, output_file):
        with open(output_file, 'w', encoding='utf-8') as f:
            for row in data:
                f.write(json.dumps(row, ensure_ascii=False) + '\n')

    # Format 1: Save input, output, summary
    format1_data = [
        {
            "input": row["input"],
            "output": row["output"],
            "summary": row["summary"]
        }
        for _, row in df.iterrows()
    ]
    write_jsonl(format1_data, file1)

    # Format 2: Save instruction, input (formatted), output
    format2_data = [
        {
            "instruction": row["instruction"],
            "input": f"#摘要#{row['summary']}\n#原文#{row['input']}",
            "output": row["output"]
        }
        for _, row in df.iterrows()
    ]
    write_jsonl(format2_data, file2)

    # Format 3: Save output, summary, report (formatted)
    format3_data = [
        {
            "output": row["output"],
            "summary": row["summary"],
            "report": f"#摘要#{row['summary']}\n#原文#{row['input']}"
        }
        for _, row in df.iterrows()
    ]
    write_jsonl(format3_data, file3)

    print("Files saved:")
    print(f"Format 1: {file1}")
    print(f"Format 2: {file2}")
    print(f"Format 3: {file3}")

In [ ]:
# prompt_version = prompt_file.split("_")[-1].split(".")[0]
# test_target_file = f'/root/autodl-tmp/data/target_data/prompt_{prompt_version}_qwen25_14B/yanbao_test_llama.json'
# train_target_file = f'/root/autodl-tmp/data/target_data/prompt_{prompt_version}_qwen25_14B/yanbao_train_llama.json'
prompt_version = prompt_file.split("_")[-1].split(".")[0]
test_target_file = f'/root/autodl-tmp/data/target_data/prompt_{prompt_version}_qwen25_14B/yanbao_test_thuc.json'
train_target_file = f'/root/autodl-tmp/data/target_data/prompt_{prompt_version}_qwen25_14B/yanbao_train_thuc.json'

In [ ]:
save_different_formats(test_df, test_target_file)

Files saved:
Format 1: /root/autodl-tmp/data/target_data/prompt_v6_qwen25_14B/yanbao_test_llama_raw_format.json
Format 2: /root/autodl-tmp/data/target_data/prompt_v6_qwen25_14B/yanbao_test_llama_with_instru_format.json
Format 3: /root/autodl-tmp/data/target_data/prompt_v6_qwen25_14B/yanbao_test_llama_wo_instru_format.json


In [ ]:
save_different_formats(train_df, train_target_file)

Files saved:
Format 1: /root/autodl-tmp/data/target_data/prompt_v6_qwen25_14B/yanbao_train_llama_raw_format.json
Format 2: /root/autodl-tmp/data/target_data/prompt_v6_qwen25_14B/yanbao_train_llama_with_instru_format.json
Format 3: /root/autodl-tmp/data/target_data/prompt_v6_qwen25_14B/yanbao_train_llama_wo_instru_format.json
